<a href="https://colab.research.google.com/github/ParthDhore/UCLAHack/blob/main/ucla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade --quiet  langchain langchain_experimental langchain-mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.5/202.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_experimental.tabular_synthetic_data.openai import (
    OPENAI_TEMPLATE,
    create_openai_data_generator,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)
from langchain_mistralai.chat_models import ChatMistralAI

In [ ]:
ads=pd.read_csv('/content/drive/MyDrive/ucla/train_data_ads.csv')
ads.head()

,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,...,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,373250,0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
1,373253,1,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
2,373252,1,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
3,373251,0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
4,373255,0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030328,39^220^16,0,15


In [ ]:
ads.dtypes

log_id                    int64
label                     int64
user_id                   int64
age                       int64
gender                    int64
residence                 int64
city                      int64
city_rank                 int64
series_dev                int64
series_group              int64
emui_dev                  int64
device_name               int64
device_size               int64
net_type                  int64
task_id                   int64
adv_id                    int64
creat_type_cd             int64
adv_prim_id               int64
inter_type_cd             int64
slot_id                   int64
site_id                   int64
spread_app_id             int64
hispace_app_tags          int64
app_second_class          int64
app_score               float64
ad_click_list_v001       object
ad_click_list_v002       object
ad_click_list_v003       object
ad_close_list_v001       object
ad_close_list_v002       object
ad_close_list_v003       object
pt_d    

In [ ]:
ads['u_newsCatInterestsST'].value_counts()

u_newsCatInterestsST
0                    157500
98                     7493
168                    7070
78^98^98^98^98         5875
219                    4499
                      ...  
219^80^199^76^21          1
38^39^152^20^78           1
173^195^44^167^96         1
98^98^78^147^10           1
15^109^199^21^220         1
Name: count, Length: 187576, dtype: int64

In [ ]:
class AdsModel(BaseModel):
    log_id: int
    label: int
    user_id: int
    age: int
    gender: int
    residence: int
    city: int
    city_rank: int
    series_dev: int
    series_group: int
    emui_dev: int
    device_name: int
    device_size: int
    net_type: int
    task_id: int
    adv_id: int
    creat_type_cd: int
    adv_prim_id: int
    inter_type_cd: int
    slot_id: int
    site_id: int
    spread_app_id: int
    hispace_app_tags: int
    app_second_class: int
    app_score: float
    ad_click_list_v001: str
    ad_click_list_v002: str
    ad_click_list_v003: str
    ad_close_list_v001: str
    ad_close_list_v002: str
    ad_close_list_v003: str
    pt_d: int
    u_newsCatInterestsST: str
    u_refreshTimes: int
    u_feedLifeCycle: int

In [ ]:
str(ads.iloc[0].to_dict())

"{'log_id': 373250, 'label': 0, 'user_id': 100005, 'age': 3, 'gender': 2, 'residence': 16, 'city': 147, 'city_rank': 2, 'series_dev': 32, 'series_group': 6, 'emui_dev': 35, 'device_name': 312, 'device_size': 2117, 'net_type': 7, 'task_id': 14339, 'adv_id': 19319, 'creat_type_cd': 6, 'adv_prim_id': 1846, 'inter_type_cd': 4, 'slot_id': 46, 'site_id': 1, 'spread_app_id': 162, 'hispace_app_tags': 47, 'app_second_class': 14, 'app_score': 10.0, 'ad_click_list_v001': '30157^30648^14278^31706', 'ad_click_list_v002': '2066^1776^1036', 'ad_click_list_v003': '114^219^312', 'ad_close_list_v001': '24107', 'ad_close_list_v002': '1218', 'ad_close_list_v003': '173', 'pt_d': 202206030326, 'u_newsCatInterestsST': '39^220^16', 'u_refreshTimes': 0, 'u_feedLifeCycle': 15}"

In [ ]:
examples=[]
for i in range(10):
  temp=ads.iloc[i].to_dict()
  formatted_str = ", ".join(f"{key}: {repr(value)}" for key, value in temp.items())
  examples.append({"example": str(formatted_str)})

In [ ]:
examples[0]

{'example': "log_id: 373250, label: 0, user_id: 100005, age: 3, gender: 2, residence: 16, city: 147, city_rank: 2, series_dev: 32, series_group: 6, emui_dev: 35, device_name: 312, device_size: 2117, net_type: 7, task_id: 14339, adv_id: 19319, creat_type_cd: 6, adv_prim_id: 1846, inter_type_cd: 4, slot_id: 46, site_id: 1, spread_app_id: 162, hispace_app_tags: 47, app_second_class: 14, app_score: 10.0, ad_click_list_v001: '30157^30648^14278^31706', ad_click_list_v002: '2066^1776^1036', ad_click_list_v003: '114^219^312', ad_close_list_v001: '24107', ad_close_list_v002: '1218', ad_close_list_v003: '173', pt_d: 202206030326, u_newsCatInterestsST: '39^220^16', u_refreshTimes: 0, u_feedLifeCycle: 15"}

In [ ]:
MISTRALAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=MISTRALAI_TEMPLATE,
)

In [ ]:
synthetic_data_generator = create_openai_data_generator(
    # output_schema=AdsModel,
    llm=ChatMistralAI(
        api_key='jVgK7BBoeySjseOraGsC7Dzcqp1bs7km',
        temperature=1
    ),
    prompt=prompt_template,
)

TypeError: create_openai_data_generator() missing 1 required positional argument: 'output_schema'

In [ ]:
synthetic_results = synthetic_data_generator.generate(
    subject="ads",
    extra="",
    runs=10,
)